<a href="https://colab.research.google.com/github/MattJCR/Red-neuronal-Basica/blob/master/Red_neuronal_Basica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

## Modificación de la implementación de la red neuronal

* Agrega el método weigths() a la clase Network, de tal forma que permita obtener los pesos de las neuronas

* Agrega el método set_weights() a la clase Network, de tal forma que permite definir los pesos de las neuronas
     

* Agrega los métodos sigmoide() y sigmoide_prime() a la clase Network

* Modifica la clase Network, para que se pueda decidir qué función de activación utilizar: relu() o sigmoide()
     

* Los métodos predict() y accuracy() de la clase Network están implementados para resolver problemas de clasificación

* Modifícalos de tal manera que también se puedan utilizar con problemas de regresión
     

* Modifica el método backpropagation() de tal manera que devuelva como resultado el array de valores de los nodos durante las épocas de entrenamiento
     

* Una vez implementados los cambios, entrena la red neuronal del ejemplo de los apuntes
```
examples = []
examples.append([[0.5, 0.67, 0.5], [0.25, 0.6]])
```

* Ejecuta la red neuronal para los datos de ejemplo de los apuntes

* Comprueba los valores de los nodos y de los pesos

* Los valores de los nodos tienen que ser los mismos que los de los apuntes

* Los valores de los pesos son ligeramente diferentes, ¿por qué?

```
net = Network([3, 4, 2])

net.set_weights([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
```

o

```
net.set_weights([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4], [0.5, 0.5, 0.5, 0.5], [0.6, 0.6, 0.6, 0.6]])

valores_nodos = net.backpropagation(0.9, examples, 1)

print(valores_nodos)
net.weights()
```

In [30]:
class NetNode(object):
  def __init__(self):
    self.inputs = []
    self.weights = []
    self.value = None

class Network(object):

  def __init__(self, layers, function = 'relu', prediction_type = 'classification'):
    # regression
    self.net = [[NetNode() for _ in range(size)] for size in layers]
    self.function = function
    self.prediction_type = prediction_type
    sizes = len(layers)

    for layer in range(1,sizes):
      for node in self.net[layer]:
        for unit in self.net[layer - 1]:
          node.inputs.append(unit)
          node.weights.append(0)
        node.weights = [np.random.uniform() for _ in range(len(node.weights))]

  def backpropagation(self, eta, examples, epochs):
        inputs = self.net[0]
        outputs = self.net[-1]
        layer_size = len(self.net)
        '''
        for layer in self.net[1:]:
            for node in layer:
                node.weights = [np.random.uniform() for _ in range(len(node.weights))]
        '''
        for epoch in range(epochs):
            for x_train, y_train in examples:
                for value, node in zip(x_train, inputs):
                    node.value = value
                for layer in self.net[1:]:
                    for node in layer:
                        in_val = [n.value for n in node.inputs]
                        unit_value = np.dot(in_val, node.weights)
                        if self.function == 'sigmoid':
                          node.value = self.sigmoide(unit_value)
                        else:
                          node.value = self.relu(unit_value)
                delta = [[] for _ in range(layer_size)]
                err = [y_train[i] - outputs[i].value for i in range(len(outputs))]
                if self.function == 'sigmoid':
                  delta[-1] = [self.sigmoide_prime(outputs[i].value) * err[i] for i in range(len(outputs))]
                else:
                  delta[-1] = [self.relu_prime(outputs[i].value) * err[i] for i in range(len(outputs))]
                hidden_layers = layer_size - 2
                for i in range(hidden_layers, 0, -1):
                    layer = self.net[i]
                    n_layers = len(layer)
                    w = [[node.weights[l] for node in self.net[i + 1]] for l in range(n_layers)]
                    if self.function == 'sigmoid':
                      delta[i] = [self.sigmoide_prime(layer[j].value) * np.dot(w[j], delta[i + 1]) for j in range(n_layers)]
                    else:
                      delta[i] = [self.relu_prime(layer[j].value) * np.dot(w[j], delta[i + 1]) for j in range(n_layers)]
                for i in range(1, layer_size):
                    layer = self.net[i]
                    in_val = [node.value for node in self.net[i - 1]]
                    n_layers = len(self.net[i])
                    for j in range(n_layers):
                        layer[j].weights = np.add(layer[j].weights, np.multiply(eta * delta[i][j], in_val))
            # print(f"epoch {epoch}/{epochs} | total error={np.sum(err)/len(examples)}")
  def predict(self,input_data):
    inputs = self.net[0]

    for v, n in zip(input_data, inputs):
      n.value = v

    for layer in self.net[1:]:
      for node in layer:
        in_val = [n.value for n in node.inputs]
        unit_value = np.dot(in_val, node.weights)
        if self.function == 'sigmoid':
          node.value = self.sigmoide(unit_value)
        else:
          node.value = self.relu(unit_value)
    
    outputs = self.net[-1]
    if self.prediction_type == 'regression':
      return [node.value for node in outputs]
    else:
      return outputs.index(max(outputs, key=lambda node: node.value))

  def accuracy(self, examples):
      if self.prediction_type == 'regression':
        error = 0
        for x_test, y_test in examples:
          prediction = self.predict(x_test)
          error += (prediction[0] - y_test[0])**2
        return error / len(examples)
      else:
        correct = 0
        for x_test, y_test in examples:
          prediction = self.predict(x_test)

          if(y_test[prediction] == 1):
            correct += 1
        return correct / len(examples)
  def relu(self,z):
    return max(0,z)

  def relu_prime(self, z):
    return 1 if z > 0 else 0

  def sigmoide(self, z):
    return 1/(1 + np.exp(-z))

  def sigmoide_prime(self, z):
    return z * (1 - z)
  
  def set_weights(self, layers):
    try:
      index = 0
      for layer in self.net:
        for node in layer:
          node.weights = layers[index]
          index += 1
    except:
      print("An exception occurred")

  def weights(self):
    weights = []
    if self.net != None:
      for layer in self.net:
        for node in layer:
          weights.append(node.weights)
    return weights
  

In [31]:
iris_X,iris_y = datasets.load_iris(return_X_y = True)

In [32]:
iris_X[:10]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

In [33]:
iris_y[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [34]:
iris_x_normalized = normalize(iris_X,axis=0)
iris_x_normalized[:10]

array([[0.07056264, 0.09254209, 0.02754801, 0.01150242],
       [0.06779548, 0.07932179, 0.02754801, 0.01150242],
       [0.06502832, 0.08460991, 0.02558029, 0.01150242],
       [0.06364474, 0.08196585, 0.02951572, 0.01150242],
       [0.06917906, 0.09518615, 0.02754801, 0.01150242],
       [0.07471338, 0.10311833, 0.03345115, 0.02300485],
       [0.06364474, 0.08989803, 0.02754801, 0.01725364],
       [0.06917906, 0.08989803, 0.02951572, 0.01150242],
       [0.06087757, 0.07667773, 0.02754801, 0.01150242],
       [0.06779548, 0.08196585, 0.02951572, 0.00575121]])

In [35]:
X_train, X_test, y_train, y_test = train_test_split(iris_x_normalized,iris_y,test_size=0.2, shuffle=True)

In [36]:
y_train = np_utils.to_categorical(y_train,num_classes=3)
y_test = np_utils.to_categorical(y_test,num_classes=3)

In [37]:
examples = []
for i in range(len(X_train)):
  examples.append([X_train[i],y_train[i]])

In [38]:
net = Network([4,7,3])

In [39]:
net.weights()

[[],
 [],
 [],
 [],
 [0.3036454168515097,
  0.9842437185428755,
  0.1393690131584675,
  0.6673635286107676],
 [0.2451757176210052,
  0.4244585368013345,
  0.8277995345412228,
  0.36227020119657627],
 [0.963924307893148,
  0.5767662639581619,
  0.04172308141090941,
  0.7298507276115839],
 [0.8370851836073955,
  0.9607738374194411,
  0.48031361294660935,
  0.5590488170969561],
 [0.6543906078051084,
  0.03671378020861338,
  0.020726670073533238,
  0.3937710130041049],
 [0.7084118889197618,
  0.24477363841048805,
  0.5707323847348281,
  0.38921288176250324],
 [0.3294425990307901,
  0.44319121087031,
  0.2488214893247752,
  0.30860383019357596],
 [0.4246957712662671,
  0.34724577566282644,
  0.9647905253112153,
  0.4155347167918704,
  0.39582609663894053,
  0.5725532632930269,
  0.916031978956715],
 [0.250471950864405,
  0.6036069391512584,
  0.30999361099377565,
  0.8185221138956127,
  0.7349230754276986,
  0.7987742050433763,
  0.06255354727791806],
 [0.41565027982033076,
  0.694942858976

In [40]:
layers_weights = [
    # Layer 0
    [],
    [],
    [],
    [],
    # Layer 1
    [ 0.1, -0.1 ,  0.1,  1.1 ],
    [0.1, 0.1, 1.1, 1.1],
    [ 0.1,  0.1, -0.1,  0.1 ],
    [-0.1, -0.1, -0.1,  0.1],
    [ 0.1,  1.1,  0.1, -1.1],
    [ 1.1,  1.1, -0.1, -1.1],
    [ 0.1,  1.1, -0.1,  0.1],
    # Layer 2
    [-0.1 , -0.1,  0.1,  0.1,  1.1, 2.1,  0.1],
    [ 0.1,  0.1,  0.1,  0.1,  0.1, 0.1 , -0.1],
    [ 1.1,  0.1,  0.1,  0.1, -0.1, -0.1, -0.1]
]
net.set_weights(layers_weights)
net.weights()

[[],
 [],
 [],
 [],
 [0.1, -0.1, 0.1, 1.1],
 [0.1, 0.1, 1.1, 1.1],
 [0.1, 0.1, -0.1, 0.1],
 [-0.1, -0.1, -0.1, 0.1],
 [0.1, 1.1, 0.1, -1.1],
 [1.1, 1.1, -0.1, -1.1],
 [0.1, 1.1, -0.1, 0.1],
 [-0.1, -0.1, 0.1, 0.1, 1.1, 2.1, 0.1],
 [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, -0.1],
 [1.1, 0.1, 0.1, 0.1, -0.1, -0.1, -0.1]]

In [41]:
net.backpropagation(0.03,examples,100)

In [42]:
examples = []
for i in range(len(X_test)):
  examples.append([X_test[i],y_test[i]])

In [43]:
prediction = net.predict(X_test[0])
accuracy = net.accuracy(examples)

print(f"Accuracy:  {accuracy}")
print(f"Desired output:  {y_test[0]}")
print(f"Index of output:  {prediction}")

Accuracy:  0.9
Desired output:  [0. 0. 1.]
Index of output:  2


In [44]:
net.weights()

[[],
 [],
 [],
 [],
 array([-0.1198556 , -0.72857823,  0.81480209,  2.13827144]),
 array([ 0.2549601 , -0.3630043 ,  1.86177166,  1.49961473]),
 array([ 0.12922693,  0.0569682 , -0.03479743,  0.10016299]),
 array([-0.1, -0.1, -0.1,  0.1]),
 array([ 0.67438446,  1.86789339, -0.05145457, -1.54884807]),
 array([ 1.8504835 ,  2.08649787, -0.479371  , -2.07163766]),
 array([ 0.18016227,  1.25368749, -0.23382216, -0.12725766]),
 array([-0.99592913, -1.28352289,  0.06410634,  0.1       ,  1.56420656,
         2.73268852,  0.1983615 ]),
 array([ 0.20944659,  1.02691304,  0.12033936,  0.1       ,  0.09383157,
         0.47848547, -0.19761547]),
 array([ 2.19754056,  1.02512551,  0.06610544,  0.1       , -1.63168498,
        -2.30858044, -0.63858024])]

In [45]:
#@title Prueba con sigmoid y regression

net = Network([4,7,3],function = 'sigmoid', prediction_type = 'regression')

examples = []
for i in range(len(X_train)):
  examples.append([X_train[i],y_train[i]])

net.backpropagation(0.03,examples,100)

examples = []
for i in range(len(X_test)):
  examples.append([X_test[i],y_test[i]])

prediction = net.predict(X_test[0])
print(prediction)
accuracy = net.accuracy(examples)

print(f"Accuracy:  {accuracy}")
print(f"Desired output:  {y_test[0]}")
print(f"Index of output:  {prediction}")

[0.3354692287267598, 0.34024810245656495, 0.31081903456253596]
Accuracy:  0.20759421416070237
Desired output:  [0. 0. 1.]
Index of output:  [0.3354692287267598, 0.34024810245656495, 0.31081903456253596]


In [52]:
#@title Red neuronal para los datos de ejemplo de los apuntes

net = Network([3,4,2],function = 'sigmoid', prediction_type = 'regression')
net.weights()

[[],
 [],
 [],
 [0.04686426033863489, 0.1527677154410667, 0.8634589438005187],
 [0.2745680819070627, 0.6824663226959877, 0.9673424381223975],
 [0.6111650371481129, 0.0683930991294287, 0.2253336595860468],
 [0.5069473961509483, 0.8757530048649489, 0.100760238509367],
 [0.2546803264287565,
  0.30612541702679297,
  0.8284914555834763,
  0.6210472962845434],
 [0.48659418232185825,
  0.9710026472339762,
  0.5042865474878275,
  0.9064355426210309]]

In [53]:
layers_weights = [
  # L0
  [],
  [],
  [],
  # L1
  [0.1, 0.1, 0.1],# 1
  [0.2, 0.2, 0.2], # 2
  [0.3, 0.3, 0.3], # 3
  [0.4, 0.4, 0.4], # 4
  # L2
  [0.5, 0.5, 0.5, 0.5], # 5
  [0.6, 0.6, 0.6, 0.6] # 6
]

net.set_weights(layers_weights)
net.weights()

[[],
 [],
 [],
 [0.1, 0.1, 0.1],
 [0.2, 0.2, 0.2],
 [0.3, 0.3, 0.3],
 [0.4, 0.4, 0.4],
 [0.5, 0.5, 0.5, 0.5],
 [0.6, 0.6, 0.6, 0.6]]

In [58]:

prediction = net.predict([0.5, 0.67, 0.5])
print("Valores de predicción de los apuntes: [0.768,0.808]")
print(f'Valores de predicción de mi red neuronal: [{prediction[0]:.3f},{prediction[1]:.3f}]')

Valores de predicción de los apuntes: [0.768,0.808]
Valores de predicción de mi red neuronal: [0.769,0.809]
